<a href="https://colab.research.google.com/github/GJotaJunior/Postech-IA-Fase4/blob/main/emotion_activity_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tech Challenge Fase 4 - POSTECH FIAP - IA para Devs

# Análise de Emoções e Atividades em Vídeo

## Instalação e Importação de bibliotecas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install opencv-python
%pip install tensorflow
%pip install tqdm
%pip install deepface
%pip install tf-keras
%pip install mediapipe
%pip install retina-face

In [ ]:
import cv2
from tqdm import tqdm
from deepface import DeepFace
import mediapipe as mp

## Contadores de Emoções e Atividades

In [ ]:
# Contador para cada emoção detectada
emotions_count = {
    'angry': 0,
    'fear': 0,
    'neutral': 0,
    'sad': 0,
    'disgust': 0,
    'happy': 0,
    'surprise': 0
}

# Contador para cada atividade detectada
activities_count = {
    'waving': 0,
    'raising_arms': 0,
    'anomalous_movement': 0
}

## Funções de Tradução das Emoções e Atividades

In [ ]:
# Tradução das reações para português
def emotions_translate_ptbr(emotion):
    return {
        'angry': 'Raiva',
        'fear': 'Medo',
        'neutral': 'Neutro',
        'sad': 'Triste',
        'disgust': 'Nojo',
        'happy': 'Feliz',
        'surprise': 'Surpreso'
    }[emotion]

# Tradução das atividades para português
def activities_translate_ptbr(activity):
    return {
        'waving': 'Acenando',
        'raising_arms': 'Levantando os bracos',
        'anomalous_movement': 'Movimento anômalo'
    }[activity]

## Inicialização do MediaPipe

In [ ]:
# Inicializar MediaPipe (biblioteca de visão computacional)
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
hands = mp_hands.Hands()
pose = mp_pose.Pose()

## Função Principal para Detecção de Emoções e Atividades

In [ ]:
def detect_faces_emotions_and_activities(video_path, output_path):
    # Capturar vídeo do arquivo especificado
    cap = cv2.VideoCapture(video_path)

    # Verificar se o vídeo foi aberto corretamente
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Obter propriedades do vídeo
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Definir o codec e criar o objeto VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Loop para processar cada frame do vídeo com barra de progresso
    for _ in tqdm(range(total_frames), desc="Processando vídeo"):
        # Ler um frame do vídeo
        ret, frame = cap.read()

        # Se não conseguiu ler o frame (final do vídeo), sair do loop
        if not ret:
            break

        # Analisar o frame para detectar faces e expressões
        result = DeepFace.analyze(
            frame,
            actions=["emotion"],
            enforce_detection=False,
            detector_backend="retinaface",
            silent=True,
        )

        # Processar mãos e poses com MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        hand_results = hands.process(frame_rgb)
        pose_results = pose.process(frame_rgb)

        # Iterar sobre cada face detectada pelo DeepFace
        for face in result:
            # Obter a caixa delimitadora da face
            x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']

            # Obter a emoção dominante
            dominant_emotion = face['dominant_emotion']
            emotions_count[dominant_emotion] += 1

            # Desenhar um retângulo ao redor da face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Escrever a emoção abaixo da face
            cv2.putText(frame, emotions_translate_ptbr(dominant_emotion), (x, y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

        # Detectar atividades das mãos e braços
        activity_detected = False
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                # Lógica para identificar atividades das mãos (ex: acenar)
                # Exemplo simplificado: se a mão está acima de um certo ponto
                if hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y < 0.5:
                    cv2.putText(frame, activities_translate_ptbr('waving'), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    activities_count['waving'] += 1
                    activity_detected = True

        if pose_results.pose_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            # Lógica para identificar atividades dos braços (ex: levantar os braços)
            left_wrist = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
            right_wrist = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
            if left_wrist.y < 0.5 and right_wrist.y < 0.5:
                cv2.putText(frame, activities_translate_ptbr('raising_arms'), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                activities_count['raising_arms'] += 1
                activity_detected = True

        # Lógica para incrementar o contador de frames que não foi identificado nenhuma das atividades mapeadas (acenar ou levantar os braços)
        if not activity_detected:
            cv2.putText(frame, activities_translate_ptbr('anomalous_movement'), (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            activities_count['anomalous_movement'] += 1

        # Escrever o frame processado no vídeo de saída
        out.write(frame)

    # Liberar a captura de vídeo e fechar todas as janelas
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    return total_frames

## Execução da detecção

In [ ]:
# Caminho para o arquivo de vídeo na mesma pasta do script
input_video_path = '/content/drive/MyDrive/fase4/video.mp4'
output_video_path = '/content/drive/MyDrive/fase4/output_video.mp4'

# Chamar a função para detectar emoções e detectar faces no vídeo, salvando o vídeo processado
total_frames = detect_faces_emotions_and_activities(input_video_path, output_video_path)

print("\nProcessamento do vídeo concluído.\n")

Processando vídeo: 100%|██████████| 3326/3326 [37:28<00:00,  1.48it/s]


Processamento do vídeo concluído.



# Relatório Final

## Exibir quantidade de emoções e atividades detectadas

In [ ]:
print("Quantidade de frames processados:", total_frames)

print("\nNúmero de emoções detectadas:")
for emotion, count in emotions_count.items():
    print(f"{emotions_translate_ptbr(emotion)}: {count}")
    print(f"Porcentagem de frames com {emotions_translate_ptbr(emotion)}: {count / total_frames * 100:.2f}%")

print("\nNúmero de atividades detectadas:")
for activity, count in activities_count.items():
    print(f"{activities_translate_ptbr(activity)}: {count}")
    print(f"Porcentagem de frames com {activities_translate_ptbr(activity)}: {count / total_frames * 100:.2f}%")

Quantidade de frames processados: 3326

Número de emoções detectadas:
Raiva: 409
Porcentagem de frames com Raiva: 12.30%
Medo: 957
Porcentagem de frames com Medo: 28.77%
Neutro: 1282
Porcentagem de frames com Neutro: 38.54%
Triste: 1586
Porcentagem de frames com Triste: 47.68%
Nojo: 1
Porcentagem de frames com Nojo: 0.03%
Feliz: 710
Porcentagem de frames com Feliz: 21.35%
Surpreso: 179
Porcentagem de frames com Surpreso: 5.38%

Número de atividades detectadas:
Acenando: 420
Porcentagem de frames com Acenando: 12.63%
Levantando os braços: 136
Porcentagem de frames com Levantando os braços: 4.09%
Movimento anômalo: 2819
Porcentagem de frames com Movimento anômalo: 84.76%
